In [ ]:
# Add directory above current directory to path
import sys; sys.path.insert(0, '..')

from functools import partial
from pathlib import Path
from pprint import pprint

import optuna
import pandas as pd
from IPython.display import display

from config.convolutional_features import *
from data_preparation.io_utils import read_yaml_file
from dataset.dataset_builder import DatasetBuilder
from model.model_builder import ModelBuilder
from model.model_tuning import OptunaOptimizer
from pipeline.trainer import Trainer

CONFIG_PATH = Path('../config/config.yaml')

In [ ]:
# Data Configuration
config = read_yaml_file(CONFIG_PATH)
data_config = config['data']
print(data_config)
conv_pattern = "**/convolutional.csv"

In [ ]:
dataset_builder = DatasetBuilder(features=CONV_FEATURES)
conv_dataset = dataset_builder.create_dataset(
            data_dir=Path('../training_data'),
            test_models=data_config["test_models"],
            pattern=conv_pattern,
        )

In [ ]:
print(f"Number of training samples: {len(conv_dataset.train.input_features)}")
print(f"Number of testing samples: {len(conv_dataset.test.input_features)}")

## Baseline Models


Baseline models for power and runtime models, we will use the mean of train dataset as prediction.


### Baseline Result

Test dataset RMSPE Power : 85.87%

Test dataset RMSPE Runtime: 110.50%

In [ ]:
trainer = Trainer(data_config=data_config, model_config=config['model'], features=CONV_FEATURES)

In [ ]:
mean_power = conv_dataset.train.power.mean()
target = conv_dataset.test.power

In [ ]:
pprint(trainer.eval_metrics(actual=target, pred=[mean_power]*len(target)))

In [ ]:
mean_runtime = conv_dataset.train.runtime.mean()
target = conv_dataset.test.runtime

In [ ]:
pprint(trainer.eval_metrics(actual=target, pred=[mean_runtime]*len(target)))

## Power

[Optuna](https://optuna.org/) library is used to perform hyperparameter tuning for power model.

In [ ]:
X_train = conv_dataset.train.input_features.values
y_power_train = conv_dataset.train.power.values
print(f"Training shape: {X_train.shape}, {y_power_train.shape}")

X_test = conv_dataset.test.input_features.values
y_power_test = conv_dataset.test.power.values
print(f"Testing shape: {X_test.shape}, {y_power_test.shape}")

In [ ]:
model_builder = ModelBuilder()
optimizer = OptunaOptimizer(X_train=X_train, y_train=y_power_train, X_test=X_test, y_test=y_power_test, model_builder=model_builder)

In [ ]:
%%time

# Ignore ConvergenceWarning from sklearn to avoid tab crash
from warnings import filterwarnings
filterwarnings('ignore')

# Maximize the test R^2 score during tuning
power_study = optuna.create_study(study_name='conv_power_model_tuning', direction="maximize", storage="sqlite:///conv_power_model_tuning.db")
# Run study for 100 trials
power_study.optimize(partial(optimizer.objective, 
                             features_mapping=dataset_builder.features_mapping, 
                             special_terms_list=[TOTAL_CONV_OPS_PER_INPUT, TOTAL_CONV_OPS_PER_BATCH]), 
               n_trials=100)

In [ ]:
# print the best performing pipeline
pprint(power_study.best_trial)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(power_study)

## Runtime

[Optuna](https://optuna.org/) library is used to perform hyperparameter tuning for runtime model.

In [ ]:
X_train = conv_dataset.train.input_features.values
y_runtime_train = conv_dataset.train.runtime.values
print(f"Training shape: {X_train.shape}, {y_runtime_train.shape}")

X_test = conv_dataset.test.input_features.values
y_runtime_test = conv_dataset.test.runtime.values
print(f"Testing shape: {X_test.shape}, {y_runtime_test.shape}")

In [ ]:
model_builder = ModelBuilder()
optimizer = OptunaOptimizer(X_train=X_train, y_train=y_runtime_train, X_test=X_test, y_test=y_runtime_test, model_builder=model_builder)

In [ ]:
%%time

# Ignore ConvergenceWarning from sklearn to avoid tab crash
from warnings import filterwarnings
filterwarnings('ignore')

# Maximize the test R^2 score during tuning
runtime_study = optuna.create_study(study_name='conv_runtime_model_tuning', direction="maximize", storage="sqlite:///conv_runtime_model_tuning.db")
# Run study for 100 trials
runtime_study.optimize(partial(optimizer.objective, 
                               features_mapping=dataset_builder.features_mapping, 
                               special_terms_list=[TOTAL_CONV_OPS_PER_INPUT, TOTAL_CONV_OPS_PER_BATCH]), 
               n_trials=100)

In [ ]:
# print the best performing pipeline
pprint(runtime_study.best_trial)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(runtime_study)